![](../../docs/colorful_logo.png)

# Ananke Pipeline Description
![](structure.png)

In [1]:
import os
os.environ["http_proxy"]="http://192.168.1.248:33210/"
os.environ["https_proxy"]="http://192.168.1.248:33210/"

# Structure Design


## Interaction Pipeline

### UserInterface (No in startup)

> 在初级的交互式界面中主要使用nodejs + bootstrap
 

 - [x] 任务需要跑通koodo


### Client(Local/Web)Server

服务发送以内部http request or 直接调用

#### Client API



#### Server API

### LLM Interface


LLM Support List：
- OpenAI
- Anthropic
- Gradient
- Hugging Face
- EverlyAI
- LiteLLM
- PaLM
- Predibase
- Replicate
- LangChain
- Llama API
- Llama CPP
- Xorbits Inference
- MonsterAPI
- RunGPT
- Portkey
- AnyScale
- Ollama
- Konko

### Context Processor

主要分为Context构造压缩重复存储&结构化

#### Prompt

在传统的定义模式中类似langchain & llamaindex 他们的prompt实现忽视了不同阶段的定义例子如下
```python
from llama_index.llms import ChatMessage, MessageRole
from llama_index.prompts import ChatPromptTemplate

# Text QA Prompt
chat_text_qa_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content="Always answer the question, even if the context isn't helpful.",
    ),
    ChatMessage(
        role=MessageRole.USER,
        content=(
            "Context information is below.\n"
            "---------------------\n"
            "{context_str}\n"
            "---------------------\n"
            "Given the context information and not prior knowledge, "
            "answer the question: {query_str}\n"
        ),
    ),
]
text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)
```
这种定义形式忽略了初始prompt和持续对话prompt对定义的不同需求，即初始定义为可选项，在前向对话的时候引入另一组prompt预设。当然随着历史对话的增长输出性能会随之下降
大量的情况，在固定context长度的时候这种长对话不建议超过max token

**** 
因此在框架中引入了下列新的构造形式：

因此我们可以看到这个结构充分的扩展了构造结果的适用性


#### Compressor

这类问题主要源于针对某一问题的context压缩,这一问题的主要来源是我们通常并不能从一个给定的向量数据库查询结果的文章段落中很好的突出重点信息。而和问题相关的内容通常只占相关段落的很少一部分，因此需要对context进行筛选和压缩

One challenge with retrieval is that usually you don't know the specific queries your document storage system will face when you ingest data into the system. This means that the information most relevant to a query may be buried in a document with a lot of irrelevant text. Passing that full document through your application can lead to more expensive LLM calls and poorer responses.

Contextual compression is meant to fix this. The idea is simple: instead of immediately returning retrieved documents as-is, you can compress them using the context of the given query, so that only the relevant information is returned. “Compressing” here refers to both compressing the contents of an individual document and filtering out documents wholesale.

To use the Contextual Compression Retriever, you'll need:

* a base retriever
* a Document Compressor
The Contextual Compression Retriever passes queries to the base retriever, takes the initial documents and passes them through the Document Compressor. The Document Compressor takes a list of documents and shortens it by reducing the contents of documents or dropping documents altogether.

![](compressor.jpg)

下面是一个langchain的官方案例：


```python
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS

documents = TextLoader('../../../state_of_the_union.txt').load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
retriever = FAISS.from_documents(texts, OpenAIEmbeddings()).as_retriever()

docs = retriever.get_relevant_documents("What did the president say about Ketanji Brown Jackson")
pretty_print_docs(docs)
```

```markdown
    Document 1:
    
    Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 
    
    Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 
    
    One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 
    
    And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
    ----------------------------------------------------------------------------------------------------
    Document 2:
    
    A former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since she’s been nominated, she’s received a broad range of support—from the Fraternal Order of Police to former judges appointed by Democrats and Republicans. 
    
    And if we are to advance liberty and justice, we need to secure the Border and fix the immigration system. 
    
    We can do both. At our border, we’ve installed new technology like cutting-edge scanners to better detect drug smuggling.  
    
    We’ve set up joint patrols with Mexico and Guatemala to catch more human traffickers.  
    
    We’re putting in place dedicated immigration judges so families fleeing persecution and violence can have their cases heard faster. 
    
    We’re securing commitments and supporting partners in South and Central America to host more refugees and secure their own borders.
    ----------------------------------------------------------------------------------------------------
    Document 3:
    
    And for our LGBTQ+ Americans, let’s finally get the bipartisan Equality Act to my desk. The onslaught of state laws targeting transgender Americans and their families is wrong. 
    
    As I said last year, especially to our younger transgender Americans, I will always have your back as your President, so you can be yourself and reach your God-given potential. 
    
    While it often appears that we never agree, that isn’t true. I signed 80 bipartisan bills into law last year. From preventing government shutdowns to protecting Asian-Americans from still-too-common hate crimes to reforming military justice. 
    
    And soon, we’ll strengthen the Violence Against Women Act that I first wrote three decades ago. It is important for us to show the nation that we can come together and do big things. 
    
    So tonight I’m offering a Unity Agenda for the Nation. Four big things we can do together.  
    
    First, beat the opioid epidemic.
    ----------------------------------------------------------------------------------------------------
    Document 4:
    
    Tonight, I’m announcing a crackdown on these companies overcharging American businesses and consumers. 
    
    And as Wall Street firms take over more nursing homes, quality in those homes has gone down and costs have gone up.  
    
    That ends on my watch. 
    
    Medicare is going to set higher standards for nursing homes and make sure your loved ones get the care they deserve and expect. 
    
    We’ll also cut costs and keep the economy going strong by giving workers a fair shot, provide more training and apprenticeships, hire them based on their skills not degrees. 
    
    Let’s pass the Paycheck Fairness Act and paid leave.  
    
    Raise the minimum wage to $15 an hour and extend the Child Tax Credit, so no one has to raise a family in poverty. 
    
    Let’s increase Pell Grants and increase our historic support of HBCUs, and invest in what Jill—our First Lady who teaches full-time—calls America’s best-kept secret: community colleges.
```

我们可以看到，给定一个示例问题，我们的检索器返回一两个相关文档和一些不相关的文档。甚至相关文档中也有很多不相关的信息。这对于整个内容的后续交互和生成是极其不必要的，在langchain中内置了很多压缩和过滤的模块


* LLMChainFilter

The LLMChainFilter is slightly simpler but more robust compressor that uses an LLM chain to decide which of the initially retrieved documents to filter out and which ones to return, without manipulating the document contents.

* EmbeddingsFilter

Making an extra LLM call over each retrieved document is expensive and slow. The EmbeddingsFilter provides a cheaper and faster option by embedding the documents and query and only returning those documents which have sufficiently similar embeddings to the query.

* Stringing compressors and document transformers together

Using the DocumentCompressorPipeline we can also easily combine multiple compressors in sequence. Along with compressors we can add BaseDocumentTransformers to our pipeline, which don't perform any contextual compression but simply perform some transformation on a set of documents. For example TextSplitters can be used as document transformers to split documents into smaller pieces, and the EmbeddingsRedundantFilter can be used to filter out redundant documents based on embedding similarity between documents.

Below we create a compressor pipeline by first splitting our docs into smaller chunks, then removing redundant documents, and then filtering based on relevance to the query.

```python
from langchain.document_transformers import EmbeddingsRedundantFilter
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0, separator=". ")
redundant_filter = EmbeddingsRedundantFilter(embeddings=embeddings)
relevant_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.76)
pipeline_compressor = DocumentCompressorPipeline(
    transformers=[splitter, redundant_filter, relevant_filter]
)


```

#### TempStorage

暂时性记忆存储组件是一个非常重要的领域，它对Agent得作用非常巨大，因为只依靠context window来存储历史信息的做法实质上无法支撑长线操作的agent技术。
因此tmp_storage实质上需要满足以下技术需求
- 1.对于当前任务框架下的任务状态的描述，存储，可能以COT，TOT，TOG等多种方式作为描述媒介
- 2.能够进行APIs的标准化存储的索引
- 3.能够对历史对话进行向量化和存储
    - 这点的处理流程可能会很复杂并存疑（历史信息的筛选和语义向量化存储的粒度应该是什么 or 该不该做）
- 4.对于交流对话的meta，唯一标识符，索引
- 5.对于历史信息查询后需要结构化回context得方式需要进一步的验证


The temporary memory storage component is a highly critical area with a significant impact on agents, as relying solely on a context window to store historical information is fundamentally inadequate for long-term agent technology. Therefore, tmp_storage essentially needs to meet the following technical requirements:

Description and storage of task states within the current task framework, possibly in various forms such as COT, TOT, TOG, serving as descriptive media.
Ability to standardize and index APIs for storage.
Ability to vectorize and store historical conversations.
The processing flow for this point may be complex and subject to doubts (e.g., what granularity to use for filtering historical information and semantic vectorization storage, or whether it should be done).
Unique identifiers and indexes for meta-information in communication dialogues.
Further validation is needed for the structured return to the context after querying historical information.

#### ContextStructured

## Document Pipeline

### Unified DocumentLoader

集成式的数据加载需要针对不同的数据类型和记录,以PDF为例

In [2]:
PDF1="2306.08302.pdf"
PDF2="2307.pdf"

In [3]:
from typing import List
from langchain.document_loaders.unstructured import UnstructuredFileLoader
from paddleocr import PaddleOCR
import fitz
import matplotlib.pyplot as plt
import os

class UnstructuredPaddlePDFLoader(UnstructuredFileLoader):
    """Loader that uses unstructured to load image files, such as PNGs and JPGs."""

    def _get_elements(self) -> List:
        def pdf_ocr_txt(filepath, dir_path="tmp_files"):
            full_dir_path = os.path.join(os.path.dirname(filepath), dir_path)
            if not os.path.exists(full_dir_path):
                os.makedirs(full_dir_path)
            ocr = PaddleOCR(use_angle_cls=True, lang="ch", use_gpu=False, show_log=False)
            doc = fitz.open(filepath)
            txt_file_path = os.path.join(full_dir_path, f"{os.path.split(filepath)[-1]}.txt")
            img_name = os.path.join(full_dir_path, 'tmp.png')
            with open(txt_file_path, 'w', encoding='utf-8') as fout:
                for i in range(doc.page_count):
                    page = doc[i]
                    text = page.get_text("")
                    fout.write(text)
                    fout.write("\n")

                    img_list = page.get_images()
                    for img in img_list:
                        pix = fitz.Pixmap(doc, img[0])
                        if pix.n - pix.alpha >= 4:
                            pix = fitz.Pixmap(fitz.csRGB, pix)
                        pix.save(img_name)
                        result = ocr.ocr(img_name)
                        result = list(filter(lambda x: x is not None, result))
                        ocr_result = [i[1][0] for line in result for i in line]
                        fout.write("\n".join(ocr_result))
            if os.path.exists(img_name):
                os.remove(img_name)
            return txt_file_path

        txt_file_path = pdf_ocr_txt(self.file_path)
        from unstructured.partition.text import partition_text
        return partition_text(filename=txt_file_path, **self.unstructured_kwargs)


In [4]:
loader = UnstructuredPaddlePDFLoader(PDF1, mode="elements")
docs = loader.load()
print("docs length",len(docs))

docs length 5139


----
> 从结果中我们也可以看到真对一个文档的信息提取能力。
纯OCR以及Latex文件和PDF直接解析都不足以覆盖所有场景。
因此对于文档的解析信息融合就是至关重要的问题

- 1.OCR用于提取信息区块和大致信息内容，也要使用SAM来做图像图表的检测等；
- 2.如果Latex/PostScript可用则使用信息直接融合，如果不可用则需要OCR增强
- 3.逻辑上提取的图标部分需要使用多模态/Caption任务进行Meta信息生成/提取



在此基础之上统一读取器的读取策略需要在文件名+文件头双重验证。

例如图像模态的读取需要从文件后缀
（/jpg/png...）和图片的exif文件信息互相验证

### pecific Document Spilit & Processor

文档分词组件可谓至关重要。

现有常规的如下

In [5]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
raw_documents = TextLoader('./tmp_files/2306.08302.pdf.txt').load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)
print(documents[0])

[2023-11-07 17:03:45,361] [ WARNING] text_splitter.py:187 - Created a chunk of size 4833, which is longer than the specified 1000
[2023-11-07 17:03:45,361] [ WARNING] text_splitter.py:187 - Created a chunk of size 6767, which is longer than the specified 1000
[2023-11-07 17:03:45,362] [ WARNING] text_splitter.py:187 - Created a chunk of size 3461, which is longer than the specified 1000
[2023-11-07 17:03:45,362] [ WARNING] text_splitter.py:187 - Created a chunk of size 4814, which is longer than the specified 1000
[2023-11-07 17:03:45,362] [ WARNING] text_splitter.py:187 - Created a chunk of size 4909, which is longer than the specified 1000
[2023-11-07 17:03:45,362] [ WARNING] text_splitter.py:187 - Created a chunk of size 5088, which is longer than the specified 1000
[2023-11-07 17:03:45,363] [ WARNING] text_splitter.py:187 - Created a chunk of size 5236, which is longer than the specified 1000
[2023-11-07 17:03:45,363] [ WARNING] text_splitter.py:187 - Created a chunk of size 5612, 

page_content='JOURNAL OF LATEX CLASS FILES, VOL. 14, NO. 8, AUGUST 2021\n1\nUnifying Large Language Models and\nKnowledge Graphs: A Roadmap\nShirui Pan, Senior Member, IEEE, Linhao Luo,\nYufei Wang, Chen Chen, Jiapu Wang, Xindong Wu, Fellow, IEEE\nAbstract—Large language models (LLMs), such as ChatGPT and GPT4, are making new waves in the field of natural language\nprocessing and artificial intelligence, due to their emergent ability and generalizability. However, LLMs are black-box models, which\noften fall short of capturing and accessing factual knowledge. In contrast, Knowledge Graphs (KGs), Wikipedia and Huapu for example,\nare structured knowledge models that explicitly store rich factual knowledge. KGs can enhance LLMs by providing external knowledge\nfor inference and interpretability. Meanwhile, KGs are difficult to construct and evolving by nature, which challenges the existing\nmethods in KGs to generate new facts and represent unseen knowledge. Therefore, it is complementar

In [6]:
print(documents[1])

page_content='Knowledge Graphs (KGs)\nCons:\nPros:\nImplicit Knowledge\nStructural Knowledge\nHallucination\nAccuracy\nDecisiveness\nIndecisiveness\nBlack-box\nInterpretability\nLacking Domain-\nDomain-specific Knowledge\nspecific/New Knowledge\nEvolving Knowledge\nCons:\nPros:\nIncompleteness\nGeneral Knowledge\nLacking Language\n Language Processing\nUnderstanding\nGeneralizability\nUnseen Facts\nLarge Language Models (LLMs)JOURNAL OF LATEX CLASS FILES, VOL. 14, NO. 8, AUGUST 2021\n2\nstein discovered gravity in 1687” when asked, “When did\nEinstein discover gravity?”, which contradicts the fact that\nIsaac Newton formulated the gravitational theory. This issue\nseverely impairs the trustworthiness of LLMs.\nAs black-box models, LLMs are also criticized for their\nlack of interpretability. LLMs represent knowledge implic-\nitly in their parameters. It is difficult to interpret or validate\nthe knowledge obtained by LLMs. Moreover, LLMs perform\nreasoning by a probability model, which

> 在很多层面一个固定的chunk_size和overlap实质上会浪费大量的语义。召回concat context的时候也会浪费大量空间。
因此根据语义来进行不同粒度的拆分逻辑会有更大的前景。

但首先要解决的问题是
- 0.设定最大词汇数量下，根据语义来进行分段
- 1.chunk内部的上下文关联问题
- 2.文章和chunk之间的关联问题
- 3.chunk和内部语句之间的关联问题
- 4.不同粒度之间的语义是否能够完全表征的问题




#### 因此在新的框架下我们需要真对这部分建立新的索引体系。
索引头包含了
文件级别
- 1.meta 信息的向量化
- 2.全文summary的信息的向量化
- 3.多模态信息列表以及对应的向量化
语义分段的chunk级别
- 1.语义分段的chunk向量化
- 2.根据段落的再次语义分句的向量化
- 3.chunk中的整体实体关系概念抽取
- 4.chunk中的单语句级别实体关系概念抽取，语义向量化，唯一标识符建立。
- 5.chunk中的句组语义向量化
在以上五个部分中，chunk需要进行一次总体的抽取和索引处理，chunk语义拆分后的语句还需要进行进一步的实体关系和概念抽取。
结果经过剪枝查重后，新增部分进入总体的库中
需要进行的实验设计
- 1.语义分段的语义转换性能，相比静态chunksize——overlap有多大提升
- 2.语义分段后的不同字数，转换向量质量是否有影响
- 3.多向量查询在向量数据库层面的支持情况/chroma or zilliz
- 4.不同粒度对语义特征的表征能力和空间是否一致。
....
  

Therefore, in the new framework, we need to establish a new indexing system for this part. The index header includes:

At the file level:
- 1. Vectorization of meta information.
- 2. Vectorization of full-text summary information.
- 3. List of multi-modal information and their corresponding vectorization.

At the semantic chunk level:
- 1. Vectorization of semantic chunk segments.
- 2. Vectorization based on further semantic sub-sentences within paragraphs.
- 3. Extraction of overall entity relations and concepts within the chunk.
- 4. Extraction of entity relations and concepts at the single-sentence level within the chunk, semantic vectorization, and unique identifier establishment.
- 5. Semantic vectorization of sentence groupings within the chunk.

Within these five sections, chunks require an initial extraction and indexing process, and sentences resulting from semantic chunk segmentation need further extraction of entity relations and concepts.

After pruning and deduplication, the newly added components enter the overall repository.

Experimental design is required for:
- 1. Performance comparison of semantic conversion regarding different chunk sizes and overlaps compared to static chunk sizes and overlaps.
- 2. Evaluation of the impact of different word counts after semantic chunking on vector quality.
- 3. Support for multi-vector queries at the vector database level using technologies like Chroma or Zilliz.
- 4. Assessment of the consistency of the representation capabilities and space for different granularities in semantic features.

And so on...

#### Layout Parser
在此基础上常规的读取方式存在较大的信息结构化不完整的问题：
因此需要介入Layout解析 Baseline：[repo](https://github.com/Layout-Parser/layout-parser)
引入新的内容解析策略



In [7]:
import layoutparser as lp

### MetaParser

meta信息的解析，需要和loader构成同一组件，并且需要能够反馈验证文件类型和转换策略。最好将整个过程编码化。

即将文件类型编码，通信中不展示具体的文件编码和解析策略。

## Compute System

### StructuredChunks Define

在结构化知识对大模型推理的增强过程中主要分为两种手段

1.动态知识融合
这种方法一般会要求要求LLM对知识和问题进行编码。进入到推理层，同时在attention层面进行重映射，但此类方法的设计难度较大，且通常无法在数据层面给予更大范围的支持。目前注入QAGNN类图神经网络基本无法达到较好的结果

2.查询增强的知识融合
这类方法一般会在图的遍历查询上下功夫，此种方法一般在网络外，在查询层面，将较大的图谱进行外挂处理。然后在Retrieving流程层面重新设计

在定义层面要考虑目前生成和运算的困境
1.数学表达式
2.逻辑表达式
3.实体和关系定义
4.存储结构，体系结构
5.时间空间序列

3,数据库层面需求，多向量，多字段，通过节点信息和图数据库相互索引。

考虑
1.单个实体和关系定义需不需要进行向量化计算和存储。
如果进行计算存储可能会造成规模爆炸即产生：
图的存储可能会涉及到词在无上下文的情况下进行向量化毫无意义。
可能最小的向量计算单元为关系级别
因此在这个框架下
可能是节点不需要进行响亮持久化存储和计算。
而将关系级别的词进行向量化计算。

1.在chunk阶段分词需要尽可能按照段落语义句子来拆分（不能）。因此不同长度的chunk是否可以统一进行embedding计算
2.尝试设计流程
	1，对实体节点，关系进行提取，并支持neo4j，nebula操作
	2，对关系进行重组织后进行relation和向量化计算，以及embedding后的互相索引。
	3，节点和向量化信息的存储和Retrieving设计
	4.数学表达式：
		OCR 数学表达式提取
		Latex 数学表达式
	5.重构现有的loader
		图片和图片Meta信息提取
      		图片赋予UUID
		OCR Caption 图像提取信息结构化
		meta信息结构化
	6.逻辑表达式：
		对于现有的数理逻辑推断语言进行支持，以支持更多的walframe/z3这样的推断引擎。
	7.RAG Pipeline Definition could reinforcement knowledge interaction with knowledge illustration limit in specific area or space 
在这个情况下单个chunk的relation提取就变的至关重要。并且单个chunk需要对应多个向量，需要探究向量数据库的改造，同步的在图关系侧也需要在relation层面加入对向量id的反响索引


原文语义分割级别的chunk，作为在向量体系中被一级索引的最基本单元。
设计至关重要，在整体的索引关系上大致为(暂时性定义)

1.（Class Document）文档/文件级别：
- 关系性索引唯一标识符-（UUID）
- meta
- meta embedding
- raw content

2.(Class StructuredChunk)针真对raw content得splited chunks级别

- chunk唯一标识索引（UUID）
- chunk raw content
- chunk summary content（auto meta）
- 模态标识符
- 父文档唯一标识符（UUID）
- chunk level 定制化的实体抽取结果列表
- chunk level 定制化的关系抽取结果列表
- chunk level 定制化的三元组抽取结果列表

- chunk level 定制化的逻辑表达式抽取结果列表
- chunk level 数学表达式抽取结果列表
- ...

3.(Class StructuredSentence)真对单个chunk语义分句级别

- 实体关系，如果在chunk级别的实体关系中已经出现则不添加
- 逻辑表达式，同理，需要和chunk级别做重复性和逻辑冲突性校验
- 数学表达式
- 语句向量化
- 父chunk唯一标识符（UUID）
- 父文档唯一标识符（UUID）

****


### Module

模块作为信息处理的基本组件



### Flow

流作为组装模块的最终组件负责暴漏最后的接口
，负责整个单个Pipeline的任务调度，需要对任务的计算图进行优化然后对整体执行做出拆分，多线程执行



#### Retriever

作为定制单数据查询链路的模块应当视为每种模态数据库查询的唯一模块：
例如结构化数据对应的关系型数据库，应当有独立的Retriever基类
各个数据库再进行对于结构化信息查询的关系型数据库基类进行继承实现
同理图也是。
参考实现：
[langchain retriever](https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/schema/retriever.py)
但在图的定义过程中产生了显著的区别即我们要求
`ananke/src/ananke/data.py`中定义的节点和关系分别有符号系统和语义向量系统的双重表示。这就意味着我们需要做两方面的探索
- 1.在现有技术框架下实现：
    - 1.向量对应UUID（1），存入向量数据库
    - 2.语义的description对应UUID（1），存入Graph
    - 3.图实体节点和关系都要添加对以上UUID（1）作为索引。
- 2.变更技术框架
    - 1.可以多向量索引的底座
    - 2.基于NaLLM项目的图查询语言（以）语言和向量体系兼容试验。
    - 3.基于在以上两个语言体系下附加基于唯一索引的关系数据库体系（Easy）。




## Storage System

### VectorDB
One of the most common ways to store and search over unstructured data is to embed it and store the resulting embedding vectors, and then at query time to embed the unstructured query and retrieve the embedding vectors that are 'most similar' to the embedded query. A vector store takes care of storing embedded data and performing vector search for you.
![](https://python.langchain.com/assets/images/vector_stores-9dc1ecb68c4cb446df110764c9cc07e0.jpg)

在向量存储体系中
> 如同 langchain的定义一样'langchain/libs/langchain/langchain/schema/vectorstore.py'[https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/schema/vectorstore.py](https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/schema/vectorstore.py)所展示的定义过程接口定义：

`VectorStore`的抽象基类，用于处理文本向量存储。它提供了一些基本的方法来处理文本向量，例如添加文本、删除文本、搜索文本等。以下是`VectorStore`类的主要接口和功能：
```python
1. `add_texts(texts: Iterable[str], metadatas: Optional[List[dict]] = None, **kwargs: Any) -> List[str]`: 将文本添加到向量存储中。参数`texts`是一个可迭代的字符串列表，`metadatas`是一个可选的元数据列表，`kwargs`是向量存储特定的参数。返回添加到向量存储中的文本的ID列表。

2. `delete(ids: Optional[List[str]] = None, **kwargs: Any) -> Optional[bool]`: 根据向量ID或其他条件删除文本。参数`ids`是一个可选的ID列表，`kwargs`是其他关键字参数，子类可能使用。返回一个布尔值，表示删除是否成功，如果未实现则返回`None`。

3. `adelete(ids: Optional[List[str]] = None, **kwargs: Any) -> Optional[bool]`: 异步删除文本。参数和返回值与`delete`方法相同。

4. `aadd_texts(texts: Iterable[str], metadatas: Optional[List[dict]] = None, **kwargs: Any) -> List[str]`: 异步添加文本到向量存储。参数和返回值与`add_texts`方法相同。

5. `add_documents(documents: List[Document], **kwargs: Any) -> List[str]`: 将文档添加到向量存储中。参数`documents`是一个文档列表，`kwargs`是其他关键字参数。返回添加到向量存储中的文本的ID列表。

6. `aadd_documents(documents: List[Document], **kwargs: Any) -> List[str]`: 异步将文档添加到向量存储中。参数和返回值与`add_documents`方法相同。

7. `search(query: str, search_type: str, **kwargs: Any) -> List[Document]`: 使用指定的搜索类型返回与查询最相似的文档。参数`query`是要搜索的查询字符串，`search_type`是搜索类型（可以是`similarity`或`mmr`），`kwargs`是其他关键字参数。返回一个文档列表。

8. `asearch(query: str, search_type: str, **kwargs: Any) -> List[Document]`: 异步执行`search`方法。

9. `similarity_search(query: str, k: int = 4, **kwargs: Any) -> List[Document]`: 使用相似性搜索返回与查询最相似的文档。参数`query`是要搜索的查询字符串，`k`是要返回的文档数量，默认为4，`kwargs`是其他关键字参数。返回一个文档列表。

10. `asimilarity_search(query: str, k: int = 4, **kwargs: Any) -> List[Document]`: 异步执行`similarity_search`方法。

11. `similarity_search_with_score(query: str, *args: Any, **kwargs: Any) -> List[Tuple[Document, float]]`: 使用相似性搜索返回与查询最相似的文档及其相似度分数。参数和返回值与`similarity_search`方法相同。

12. `asimilarity_search_with_score(query: str, *args: Any, **kwargs: Any) -> List[Tuple[Document, float]]`: 异步执行`similarity_search_with_score`方法。

13. `similarity_search_with_relevance_scores(query: str, k: int = 4, **kwargs: Any) -> List[Tuple[Document, float]]`: 使用相似性搜索返回与查询最相似的文档及其相关性分数。参数和返回值与`similarity_search_with_score`方法相同。

14. `asimilarity_search_with_relevance_scores(query: str, k: int = 4, **kwargs: Any) -> List[Tuple[Document, float]]`: 异步执行`similarity_search_with_relevance_scores`方法。

15. `similarity_search_by_vector(query: List[float], k: int = 4, **kwargs: Any) -> List[Document]`: 使用向量搜索返回与查询最相似的文档。参数`query`是要搜索的查询向量，`k`是要返回的文档数量，默认为4，`kwargs`是其他关键字参数。返回一个文档列表。

16. `asimilarity_search_by_vector(query: List[float], k: int = 4, **kwargs: Any) -> List[Document]`: 异步执行`similarity_search_by_vector`方法。

17. `max_marginal_relevance_search(query: str, k: int = 4, fetch_k: int = 20, lambda_mult: float = 0.5, **kwargs: Any) -> List[Document]`: 使用最大边际相关性搜索返回与查询最相关的文档。参数`query`是要搜索的查询字符串，`k`是要返回的文档数量，默认为4，`fetch_k`是要传递给MMR算法的文档数量，默认为20，`lambda_mult`是结果多样性的度量，范围为0到1，默认为0.5，`kwargs`是其他关键字参数。返回一个文档列表。

18. `amax_marginal_relevance_search(query: str, k: int = 4, fetch_k: int = 20, lambda_mult: float = 0.5, **kwargs: Any) -> List[Document]`: 异步执行`max_marginal_relevance_search`方法。

19. `max_marginal_relevance_search_by_vector(query: List[float], k: int = 4, fetch_k: int = 20, lambda_mult: float = 0.5, **kwargs: Any) -> List[Document]`: 使用向量搜索返回与查询最相关的文档。参数和返回值与`max_marginal_relevance_search`方法相同。

20. `amax_marginal_relevance_search_by_vector(query: List[float], k: int = 4, fetch_k: int = 20, lambda_mult: float = 0.5, **kwargs: Any) -> List[Document]`: 异步执行`max_marginal_relevance_search_by_vector`方法。

21. `from_documents(cls: Type[VST], documents: List[Document], embedding: Embeddings, **kwargs: Any) -> VST`: 从文档和嵌入初始化`VectorStore`。参数`documents`是一个文档列表，`embedding`是嵌入对象，`kwargs`是其他关键字参数。返回一个`VectorStore`实例。

22. `afrom_documents(cls: Type[VST], documents: List[Document], embedding: Embeddings, **kwargs: Any) -> VST`: 异步执行`from_documents`方法。

23. `from_texts(cls: Type[VST], texts: List[str], embedding: Embeddings, metadatas: Optional[List[dict]] = None, **kwargs: Any) -> VST`: 从文本和嵌入初始化`VectorStore`。参数`texts`是一个字符串列表，`embedding`是嵌入对象，`metadatas`是一个可选的元数据列表，`kwargs`是其他关键字参数。返回一个`VectorStore`实例。

24. `afrom_texts(cls: Type[VST], texts: List[str], embedding: Embeddings, metadatas: Optional[List[dict]] = None, **kwargs: Any) -> VST`: 异步执行`from_texts`方法。

25. `_get_retriever_tags() -> List[str]`: 获取检索器的标签。返回一个字符串列表。

26. `as_retriever(**kwargs: Any) -> VectorStoreRetriever`: 返回一个`VectorStoreRetriever`实例，该实例从当前`VectorStore`初始化。参数`kwargs`是其他关键字参数。返回一个`VectorStoreRetriever`实例。

此外，代码中还定义了一个名为`VectorStoreRetriever`的基类，用于处理`VectorStore`的检索。它继承自`BaseRetriever`类，并提供了一些用于检索文档的方法。
```


存在的问题
- 1.不支持多向量构建体系，在数据库层面的联系需要更紧密
- 2.异步执行被过度的接口化了，本质上这个需要在接口层面无感，或参数化
- 3.抽象逻辑不合理在之后的向量化存储中，需要兼顾短时和持久化记忆的双重存储需求因此需要在基础类的基础上延伸新的短期组件和长期组件。
- 4.对应大的结构化chunk数据结构这个存储接口也需要大改





#### Chroma

### Graph(Structure)DB

#### Neo4j

存储过程可以套用 llamaindex [neo4j](https://docs.llamaindex.ai/en/stable/examples/index_structs/knowledge_graph/Neo4jKGIndexDemo.html)
语义图的查询语言交互可以借鉴[NaLLM](https://github.com/neo4j/NaLLM)

In [8]:
from llama_index.graph_stores import Neo4jGraphStore
from llama_index.storage.storage_context import StorageContext
from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    KnowledgeGraphIndex,
    LLMPredictor,
    ServiceContext,
)
username = "neo4j"
password = "retractor-knot-thermocouples"
url = "bolt://44.211.44.239:7687"
database = "neo4j"
# graph_store = Neo4jGraphStore(
#     username=username,
#     password=password,
#     url=url,
#     database=database,
# )

# storage_context = StorageContext.from_defaults(graph_store=graph_store)

# # NOTE: can take a while!
# index = KnowledgeGraphIndex.from_documents(
#     documents,
#     storage_context=storage_context,
#     max_triplets_per_chunk=2,
#     service_context=service_context,
# )

#### Nebula

同理，这也就要求在上层需要保持抽象的一致性

### RelationDB

参考实现[Langchain Base Storage](https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/schema/storage.py)

关系型数据库的基类实现没有什么特别的trick，
主要聚焦于基类对配置的继承 具体实现应当从：
`ananke/src/ananke/db/__init__.py`开始继承配置项然后开始

#### Sqlite3

In [ ]:
同理

#### MariaDB

In [ ]:
同理

# Final Interaction Pipeline